# head

In [1]:
%cd ~/Workspace/kaggle-2019Q3-cellular/
!nvidia-smi

/home/chengjiun/Workspace/kaggle-2019Q3-cellular
Mon Jul  8 12:40:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   61C    P0    84W / 250W |    286MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                           

In [2]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import torchvision
from torchvision import transforms as T

import tqdm

import warnings
warnings.filterwarnings('ignore')

# setup

In [8]:
path_data = '../DATA/kaggle-2019Q3-cellular/'
device = 'cuda'
batch_size = 8

from preprocess import ImageDS
from utils import accuracy

class ImagesDS(D.Dataset):
    def __init__(self, csv_file, img_dir, mode='train', site=1, channels=[1, 2, 3, 4, 5, 6]):

        df = pd.read_csv(csv_file)
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]

    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            return T.ToTensor()(img)

    def _get_img_path(self, index, channel):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir, self.mode, experiment, f'Plate{plate}', f'{well}_s{self.site}_w{channel}.png'])

    def __getitem__(self, index):
        paths = [self._get_img_path(index, ch) for ch in self.channels]
        img = torch.cat([self._load_img_as_tensor(img_path)
                         for img_path in paths])

        if self.mode == 'train':
            return img, self.records[index].sirna
        else:
            return img, self.records[index].id_code

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

class DensNet(nn.Module):
    def __init__(self, num_classes=1000, num_channels=6):
        super().__init__()
        preloaded = torchvision.models.densenet121(pretrained=True)
        self.features = preloaded.features
        self.features.conv0 = nn.Conv2d(num_channels, 64, 7, 2, 3)
        self.classifier = nn.Linear(1024, num_classes, bias=True)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out

class MyResNet(nn.Module):
    def __init__(self, num_classes=1000, num_channels=6):
        super().__init__()
        model = torchvision.models.resnet50(pretrained=True)
        model.fc = nn.Linear(2048, num_classes, bias=True)
        # freeze parameters     
        self.model = model

    def freeze_parameters(self):
        for param in self.model.parameters():
            param.requres_grad = False
        
        for param in self.model.layer4.parameters():
            param.requires_grad = True
        for param in self.model.avgpool.parameters():
            param.requires_grad = True
        for param in self.model.fc.parameters():
            param.requires_grad = True
    
        
    def forward(self, x):
        return self.model(x)

    
@torch.no_grad()
def prediction(model, loader):
    preds = np.empty(0)
    for x, _ in loader: 
        x = x.to(device)
        output = model(x)
        idx = output.max(dim=-1)[1].cpu().numpy()
        preds = np.append(preds, idx, axis=0)
    return preds



## input

In [4]:
classes = 1108
#model = MyResNet(num_classes=classes)
#model.freeze_parameters()
model = DensNet(num_classes=classes)
model.to(device)

DensNet(
  (features): Sequential(
    (conv0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (c

In [5]:
ds = ImagesDS(path_data+'/train.csv', path_data)
ds_test = ImagesDS(path_data+'/test.csv', path_data, mode='test')

# train model

In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
epochs = 30

loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2)
tloader = D.DataLoader(ds_test, batch_size=batch_size,
                       shuffle=False, num_workers=2)

tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    for x, y in loader: 
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output, device=device)
        target[np.
               arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

Epoch 1 -> Train Loss: 0.0048, ACC: 22.23%
Epoch 2 -> Train Loss: 0.0045, ACC: 26.54%
Epoch 3 -> Train Loss: 0.0043, ACC: 29.27%
Epoch 4 -> Train Loss: 0.0042, ACC: 31.92%
Epoch 5 -> Train Loss: 0.0040, ACC: 34.70%
Epoch 6 -> Train Loss: 0.0039, ACC: 37.37%
Epoch 7 -> Train Loss: 0.0037, ACC: 40.91%
Epoch 8 -> Train Loss: 0.0036, ACC: 43.05%
Epoch 9 -> Train Loss: 0.0034, ACC: 45.62%
Epoch 10 -> Train Loss: 0.0033, ACC: 48.50%
Epoch 11 -> Train Loss: 0.0031, ACC: 51.17%
Epoch 12 -> Train Loss: 0.0030, ACC: 53.69%
Epoch 13 -> Train Loss: 0.0028, ACC: 56.27%
Epoch 14 -> Train Loss: 0.0027, ACC: 58.96%
Epoch 15 -> Train Loss: 0.0026, ACC: 61.64%
Epoch 16 -> Train Loss: 0.0025, ACC: 63.18%
Epoch 17 -> Train Loss: 0.0023, ACC: 66.13%
Epoch 18 -> Train Loss: 0.0022, ACC: 68.17%
Epoch 19 -> Train Loss: 0.0021, ACC: 70.14%
Epoch 20 -> Train Loss: 0.0020, ACC: 72.87%
Epoch 21 -> Train Loss: 0.0019, ACC: 74.59%
Epoch 22 -> Train Loss: 0.0018, ACC: 76.26%
Epoch 23 -> Train Loss: 0.0017, ACC: 78.5

# prediction test

In [9]:
preds = prediction(model, tloader)

In [10]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = preds.astype(int)
submission.to_csv('submission.2.densenet_deep.csv', index=False, columns=['id_code','sirna'])